In [2]:
%%time
import numpy as np
import pandas as pd
import os
os.chdir('E:\lqbf')
Cell_Info=pd.read_excel('小区信息.xlsx',encoding="gbk",dtype={'本地小区标识':str,'下行频点':str,'物理小区标识':str,'跟踪区域码':str})
EutranExter=pd.read_csv('查询EUTRAN外部小区.csv',encoding="gbk",usecols=['NAME','基站标识','小区标识','下行频点','上行频点配置指示', '上行频点', '物理小区标识', '跟踪区域码'],dtype={'下行频点':str,'物理小区标识':str,'跟踪区域码':str})
EutranExter['ECI']=EutranExter['基站标识']*256+EutranExter['小区标识']
EutranExter.head()
# EutranExter1=pd.merge(EutranExter,Cell_Info,how='left',left_on=EutranExter.ECI,right_on=Cell_Info.ECI)
EutranExter1=pd.merge(EutranExter,Cell_Info,how='left',on='ECI')

EutranExter1.dropna(subset=['本地小区标识'],inplace=True)

def Fre(a,b):
    if a==b:
        return '频点正确'
    else:
        return '频点错误'
def Pci(a,b):
    if a==b:
        return 'PCI正确'
    else:
        return 'PCI错误'
def Tac(a,b):
    if a==b:
        return 'Tac正确'
    else:
        return 'Tac错误'
EutranExter1['频点是否正常']=EutranExter1.apply(lambda x : Fre(x['下行频点_x'],x['下行频点_y']),axis=1)
EutranExter1['Pci是否正常']=EutranExter1.apply(lambda x : Pci(x['物理小区标识_x'],x['物理小区标识_y']),axis=1)
EutranExter1['Tac是否正常']=EutranExter1.apply(lambda x : Tac(x['跟踪区域码_x'],x['跟踪区域码_y']),axis=1)
EutranExter1['外部核查情况']=EutranExter1['频点是否正常']+EutranExter1['Pci是否正常']+EutranExter1['Tac是否正常']

EutranExter2=EutranExter1[EutranExter1['外部核查情况'].str.contains('错误')]

def mml(a,b,c,d,e,f,g):
    if g == '频点错误PCI错误Tac错误':
        return f"""MOD EUTRANEXTERNALCELL:MCC="460",MNC="00",ENODEBID={b},CELLID={c},DLEARFCN={d},PHYCELLID={e},TAC={f};{{{a}}}"""
    elif g== '频点错误PCI错误Tac正确':
        return f"""MOD EUTRANEXTERNALCELL:MCC="460",MNC="00",ENODEBID={b},CELLID={c},DLEARFCN={d},PHYCELLID={e};{{{a}}}"""
    elif g == '频点错误PCI正确Tac错误':
        return f"""MOD EUTRANEXTERNALCELL:MCC="460",MNC="00",ENODEBID={b},CELLID={c},DLEARFCN={d},TAC={f};{{{a}}}"""
    elif g == '频点错误PCI正确Tac正确':
        return f"""MOD EUTRANEXTERNALCELL:MCC="460",MNC="00",ENODEBID={b},CELLID={c},DLEARFCN={d};{{{a}}}"""
    elif g == '频点正确PCI错误Tac错误':
        return f"""MOD EUTRANEXTERNALCELL:MCC="460",MNC="00",ENODEBID={b},CELLID={c},PHYCELLID={e},TAC={f};{{{a}}}"""
    elif g == '频点正确PCI错误Tac正确':
        return f"""MOD EUTRANEXTERNALCELL:MCC="460",MNC="00",ENODEBID={b},CELLID={c},PHYCELLID={e};{{{a}}}"""
    else:
        return f"""MOD EUTRANEXTERNALCELL:MCC="460",MNC="00",ENODEBID={b},CELLID={c},TAC={f};{{{a}}}""" 

EutranExter2['MML']=EutranExter2.apply(lambda x : mml(x['NAME_x'],x['基站标识'],x['小区标识_x'],x['下行频点_y'],x['物理小区标识_y'],x['跟踪区域码_y'],x['外部核查情况']),axis=1)

EutranExter2.to_excel('EutranExter_外部核查结果.xlsx',index=None)

# EutranExter['PCI混淆情况']=EutranExter['NAME'].map(str)+"_"+EutranExter['下行频点'].map(str)+"_"+EutranExter['物理小区标识'].map(str)

# Obfuscator=pd.merge(EutranExter,pd.pivot_table(EutranExter,values='NAME',index='PCI混淆情况',aggfunc='count'),how='left',on='PCI混淆情况')

# Obfuscator.rename(columns={'NAME_y':'PCI混淆数','NAME_x':'NAME'},inplace=True)

# Obfuscator_final=Obfuscator[Obfuscator['PCI混淆数']>=2]

# Obfuscator_final.to_excel('PCI混淆核查结果.xlsx',index=False)




MemoryError: Unable to allocate array with shape (9, 2838400) and data type object

In [8]:
con1=Cell_Info['小区名称'].str.contains('HEHH') | Cell_Info['小区名称'].str.contains('HFHH') | Cell_Info['小区名称'].str.contains('HLH')

In [9]:
Cell_Info=Cell_Info[con1]

In [10]:
con2=~Cell_Info['小区名称'].str.contains('BK')&~Cell_Info['小区名称'].str.contains('拉远')&~Cell_Info['小区名称'].str.contains('EM')

In [11]:
Cell_Info=Cell_Info[con2]

In [12]:
Cell_Info['Mod3']=Cell_Info['物理小区标识'].map(int) % 3

Cell_Info['Enb_Fre_Mod3']=Cell_Info['eNodeB标识'].map(str)+"_"+Cell_Info['下行频点'].map(str)+"_"+Cell_Info['Mod3'].map(str)

Cell_Info_M=pd.merge(Cell_Info,pd.pivot_table(Cell_Info,values='下行频点',index='Enb_Fre_Mod3',aggfunc='count'),how='left',on='Enb_Fre_Mod3')

Cell_Info_M.rename(columns={'下行频点_y':'是否Mod3'},inplace=True)

Cell_Info_M_final = Cell_Info_M[Cell_Info_M['是否Mod3']>=2]

Cell_Info_M_final.to_excel('同站Mod3核查结果.xlsx',index=False)